In [3]:
# Import libraries
import time
import re
from bs4 import BeautifulSoup as soup
import csv
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as expected
from selenium.webdriver.support.ui import WebDriverWait
from selenium import webdriver
from urllib.error import HTTPError
# import random

In [89]:
def get_job_urls(job_title, page_number):
    job_title.replace(' ', '+')
    # url for Toronto job posing
    url_gta = []
    for page in range(0, 10):
        url_page = r'https://ca.indeed.com/jobs?q=' + job_title + \
            r'&l=Greater+Toronto+Area,+ON&start=' + str(page*10)
        url_gta.append(url_page)
    return url_gta

In [90]:
def get_page_content(url, job_list=True):
    """
    url - base url to access desired course list page
    """
    try:
        profile = webdriver.FirefoxProfile()
        # 1 - Allow all images
        # 2 - Block all images
        # 3 - Block 3rd party images
        profile.set_preference("permissions.default.image", 2)
        driver = webdriver.Firefox(
            executable_path=r'/home/xinda/insight/notebooks/geckodriver', firefox_profile=profile)
        driver.get(url)
        if job_list == True:
            time.sleep(2)
        else:
            try:
                wait = WebDriverWait(driver, timeout=2)
                wait.until(expected.presence_of_all_elements_located(
                    (By.TAG_NAME, 'a')))
                time.sleep(randint(1, 3))
            except:
                pass
        html = driver.page_source
        page_content = soup(html, 'html.parser')
        driver.close()  # Closed the browser opened in each loop.

        return page_content

    except HTTPError as e:
        print(e)

In [91]:
def get_job_description(job_content):
    job_description = []
    job_description_temp = job_content.find_all(
        'li')
    for link in job_description_temp:
        job_description.append(link.get_text())
    return job_description

In [95]:
def scrap_indeed(csvfile='indeed.csv', urls='url_gta'):
    root_url = 'https://ca.indeed.com/'
    """
    Scrap indeed job postings.
    Save the result in csvfile.
    """
    with open(csvfile, 'w') as indeed:
        indeed_writer = csv.writer(indeed, delimiter=',')
        indeed_writer.writerow([
            'job_title', 'job_description'
        ])  # Generate the header of csvfile.

        # loop through all the pages
        for url in urls:
            list_content = get_page_content(url, job_list=True)
            # Navigate to list of job postings
            # Parse Data
            jobs = list_content.find_all('div', {'class': 'title'})[3:] # First three are ads
            for job in jobs:
                try:
                    job_href = job.a.get('href')
                    job_title = job.a.get('title')
                    job_link = root_url+job_href
                    # Base job_link to get job description
                    job_content = get_page_content(
                    url=job_link, job_list=False)
                    job_description = get_job_description(job_content)
                    print([
                        job_title, job_description
                    ])
                        # Write into the file
                    indeed_writer.writerow([
                        job_title, job_description
                    ])
                except AttributeError as e:
                    print(e)

In [99]:
url_gta = get_job_urls('Data Scientist', 10)
url_gta

['https://ca.indeed.com/jobs?q=Data Scientist&l=Greater+Toronto+Area,+ON&start=0',
 'https://ca.indeed.com/jobs?q=Data Scientist&l=Greater+Toronto+Area,+ON&start=10',
 'https://ca.indeed.com/jobs?q=Data Scientist&l=Greater+Toronto+Area,+ON&start=20',
 'https://ca.indeed.com/jobs?q=Data Scientist&l=Greater+Toronto+Area,+ON&start=30',
 'https://ca.indeed.com/jobs?q=Data Scientist&l=Greater+Toronto+Area,+ON&start=40',
 'https://ca.indeed.com/jobs?q=Data Scientist&l=Greater+Toronto+Area,+ON&start=50',
 'https://ca.indeed.com/jobs?q=Data Scientist&l=Greater+Toronto+Area,+ON&start=60',
 'https://ca.indeed.com/jobs?q=Data Scientist&l=Greater+Toronto+Area,+ON&start=70',
 'https://ca.indeed.com/jobs?q=Data Scientist&l=Greater+Toronto+Area,+ON&start=80',
 'https://ca.indeed.com/jobs?q=Data Scientist&l=Greater+Toronto+Area,+ON&start=90']

In [112]:
job_list = ['Graphic Designers', 'Data Scientist', 'Security Engineer', 'Pharmacist', 'Marketing Specialist', 'Business Analyst',
            'Accountant', 'Customer Care Rep', 'Biologist', 'Librarian', 'Entrepreneur', 'Teacher', 'Photographer']

In [113]:
for job in job_list:
    url_gta = get_job_urls(job, 10)
    scrap_indeed(csvfile=job, urls=url_gta)

['Junior Graphic Designer', ['Find Jobs', 'Company Reviews', 'Find Salaries', 'Find Resumes', 'Employers / Post Job', 'Upload your resume', 'Sign in', 'Produce packaging design and marketing collateral for a suite of products', 'Provide conceptual, design, and copy direction as appropriate for product packaging', 'Maintain a consistent brand identity/experience across a lineup of products', 'Create and assemble comps/mock-ups for presentations and production.', 'Work collaboratively with the Product Development and Marketing teams to produce innovative designs for full size, and seasonal products', 'Support web development and social media campaigns with engaging images and content.', 'Concept, create and deliver strong design solutions across various branded materials including but not limited to email, website content, digital ads, landing pages, event collateral, signage, window graphics, and light visual merchandising work when needed', 'Maintaining and further elevating the brand 

['Graphic Designer', ['Find Jobs', 'Company Reviews', 'Find Salaries', 'Find Resumes', 'Employers / Post Job', 'Upload your resume', 'Sign in', '\nConsult with senior staff regarding marketing strategy and promotions.', '\nCreate marketing materials that are on target with marketing strategy for Sunwing Travel Group and its companies', '\nProduction of branded marketing materials, that include advertising, promotional flyers.', '\nPolice and maintain marketing campaign guidelines across all departments.', '\nMaintain contact information with all newspaper and publications including up-to-date delivery instructions and convey to rest of the team.', '\nEnsure on time delivery of all advertising materials to newspapers and publications.', '\nMaintain marketing campaign consistency and brand differentiation', '\nExpert with Adobe InDesign, Illustrator, Photoshop, Acrobat on MAC platforms.', '\nAdaptable to work independently, set and prioritize projects to meet deadlines in a fast paced en

['Junior Graphic Designer', ['Find Jobs', 'Company Reviews', 'Find Salaries', 'Find Resumes', 'Employers / Post Job', 'Upload your resume', 'Sign in', 'Produce packaging design and marketing collateral for a suite of products', 'Provide conceptual, design, and copy direction as appropriate for product packaging', 'Maintain a consistent brand identity/experience across a lineup of products', 'Create and assemble comps/mock-ups for presentations and production.', 'Work collaboratively with the Product Development and Marketing teams to produce innovative designs for full size, and seasonal products', 'Support web development and social media campaigns with engaging images and content.', 'Concept, create and deliver strong design solutions across various branded materials including but not limited to email, website content, digital ads, landing pages, event collateral, signage, window graphics, and light visual merchandising work when needed', 'Maintaining and further elevating the brand 

['Production Artist II', ['Find Jobs', 'Company Reviews', 'Find Salaries', 'Find Resumes', 'Employers / Post Job', 'Upload your resume', 'Sign in', 'Work closely with the Creative Director, Art Directors, Producers and Account Managers to ensure the latest production techniques are being utilized and all final deliverables are met.', '\nDeliver final artwork to the highest quality standards for all TD Lines of Business for tactics including, but not limited to, printed posters, direct mail, card art, display banners, digital signage and social media posts.', '\nCreate versions, execute revisions and manage other languages when required.', '\nEnsure that North American Brand Guidelines are consistently applied and when challenges with application occur help to find solutions for your own projects and on behalf of your team in partnership with Brand Governance.', '\nAct as an ambassador for the Marketing department objectives and point of view on corporate brand and its visual identity.'

['Graphic Designer', ['Find Jobs', 'Company Reviews', 'Find Salaries', 'Find Resumes', 'Employers / Post Job', 'Upload your resume', 'Sign in', 'Create daily social media photo and video content for businesses', 'Work independently to create case studies, social media content and web ads', 'Video production and editing', 'Contribute out-of-the-box ideas and translate the brand image into stunning visuals', 'Researching industry-related topics', 'Photoshop expertise', 'Adobe Premiere, After Effects, or other video editing software', 'Understanding of YouTube, Facebook, Instagram, etc.', 'Thorough understanding of file formats and sizes to optimize for web viewing', 'Understanding of following brand guidelines', '3 years as a Content Creator', 'Adobe Photoshop: 5 years', 'Adobe Premiere/After FX: 4 years', 'Photography: 3 years', 'Video editing: 2 years', 'Portfolio required', 'About', 'Help Centre']]
['Graphic Designer', ['Find Jobs', 'Company Reviews', 'Find Salaries', 'Find Resumes', 

...

['Full Stack Engineer', ['Find Jobs', 'Company Reviews', 'Find Salaries', 'Find Resumes', 'Employers / Post Job', 'Upload your resume', 'Sign in', '\nPlaying a major role in architecting frontend / backend frameworks', '\nDeveloping major application features for the platform', '\nHelping to keep the codebase maintainable, extensible and well tested', '\nImplementing standards and practices to improve and maintain code quality', '\nWorking with designers and other team members to understand our users and help guide product decisions', '\nTaking ownership of a major part of the codebase and shipping code constantly!', '\nA smart and driven developer that is motivated to build a great codebase', '\nGreat communication skills and a desire to collaborate with the team', '\nDeep experience with building web applications', '\nExperience and understanding of some part of the tech stack (Python, Javascript, AngularJS, React, Google App Engine, Django, and more)', '\nTechnical degree (though br

['Design Director', ['Find Jobs', 'Company Reviews', 'Find Salaries', 'Find Resumes', 'Employers / Post Job', 'Upload your resume', 'Sign in', 'Communicate the creative approach, process, value and thinking to clients', 'Concept, develop and manage a “best in class” creative product', 'Manage creative team, project planning and work flow', 'Direct the creative team from concept to completion', 'Plan and estimate highly complex projects', 'Handle conflict management and resolution', 'Oversee the proper application of brand systems, typography, photography, colour theory and other components to create innovative visual solutions that meet customer, business and user need', 'Provide quality assurance of all creative deliverables', 'Contribute to the development of the creative team’s culture and quality standards', 'Delegate day-to-day responsibility to creative team and outside vendors as required', 'Elevate the creative level of the work coming out of the creative team', 'Educate projec

['Dealer Services Representative - Automotive Photographer', ['Find Jobs', 'Company Reviews', 'Find Salaries', 'Find Resumes', 'Employers / Post Job', 'Upload your resume', 'Sign in', 'Working closely with automotive and non-automotive clients; staging and photographing their inventory.', 'Manage a territory of designated dealers conducting weekly capture / service visits', 'Liaison with sales & internal staff to resolve dealer admin problems and inquiries while establishing and maintaining efficient data and information flow.', 'Maintain new and existing clients through planned individual account support.', 'Reporting daily and weekly activity to management on assigned dealerships', 'Ability to perform and complete their duties in all types of weather.', 'Flexibility and adaptability to work in a changing fast paced environment', 'A valid driver’s license with an excellent driving record; Able to operate a manual transmission vehicle.', 'Knowledge of vehicles and/or the automotive ind

['Art Director', ['Find Jobs', 'Company Reviews', 'Find Salaries', 'Find Resumes', 'Employers / Post Job', 'Upload your resume', 'Sign in', 'You take initiative, seeking out and owning opportunities to improve yourself, your team, and our business.', 'You’re comfortable in a fast-paced environment, maintaining a keen attention to detail even when under pressure.', 'You thrive in a collaborative environment, sharing your knowledge with your colleagues, and celebrating success as a team.', 'You’re a life-long learner, staying current on market events, industry news, and innovative trends and technologies.', 'You believe in sustainable financial strength, and you’re committed to long-term wealth solutions.\n', 'You consider the impact of your actions on your team members, our partners, our clients, and the places where we live and work.', 'Using the brand standards and creative direction provided by the agency, the art director establishes the conceptual and stylistic direction of design 

['Real Estate Assistant', ['Find Jobs', 'Company Reviews', 'Find Salaries', 'Find Resumes', 'Employers / Post Job', 'Upload your resume', 'Sign in', 'Creating flyers/ feature sheets and organizing distribution of marketing materials', 'Managing social media advertising', 'Updating social media and websites with new listings', 'Preparing and typing offers/ amendments/ waivers using Dotloop or WebForms', 'Booking appointments and tracking confirmations – following up where required', 'Co-ordinating listing property set-up and preparation, such as booking / coordinating / scheduling photographers', 'Co-ordinating or installing/ removing signs and lockboxes', 'Co-ordinating schedules', 'Data entry', 'Expense-tracking', 'Reviewing mail, keeping the office tidy and organized', 'Assisting in the preparation of comparative market analysis (CMAs) reports', 'Updating client information and managing client database', 'Managing client relationship tasks such as following up and ensuring all requir